In [ ]:
import numpy as np
import pandas as pd

### 0. 주어진 데이터를 불어온 후 상위 10개 행을 출력해 주세요.

In [ ]:
df = pd.read_csv('../train.csv')

In [ ]:
df.head(10)

### 1. 각 Column별로 null값이 있는지 확인해주세요.

In [ ]:
df.isnull().sum()

### 2. grade Column에는 어떤 값이 있는지 확인해주세요.

In [ ]:
df['grade'].unique()

### 3. zipcode에 대해서 문자열로 바꿔주세요.

In [ ]:
df['zipcode'] = df['zipcode'].astype(object)

In [ ]:
df.info()

### 4. loc를 활용하여 date, price, grade만 출력해주세요.

In [ ]:
df.loc[:,['date','price','grade']]

### 5. grade가 13인 행들만 보여주세요.

In [ ]:
df[df['grade']==13]

### 6. grade별로 평균 가격을 보여주세요.

In [ ]:
df.groupby('grade')['price'].agg([('grade_mean','mean')])

### 7. 6번에서 만든 데이터를 원본 데이터와 merge해주세요.

In [ ]:
grade_mean = df.groupby('grade')['price'].agg([('grade_mean','mean')])
pd.merge(df, grade_mean, on='grade',how='outer')

### 8. Date Column을 이용해 년, 월, 일에 해당하는 3개의 새로운 Column을 만들어주세요.

In [ ]:
df['year'] = df['date'].apply(lambda x: x[0:4])
df['month'] = df['date'].apply(lambda x: x[4:6])
df['day'] = df['date'].apply(lambda x: x[0:4])

In [ ]:
df

### 9. 이후 Price를 예측하는 모델을 제작하기 위해 추가적인 변수를 만드려고 합니다. Price의 데이터 분포를 확인하고 1개 이상의 EDA를 통해 의견을 제시해주세요.

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

plt.rc('font', family='malgun gothic')
plt.rc('axes', unicode_minus=False)

In [ ]:
plt.title('Price Distplot')
sns.distplot(df['price'])
plt.show()

In [ ]:
log_price = np.log1p(df['price'])
sns.distplot(log_price)
plt.show()

- target값인 price의 분포가 왜도를 띄고있기에 정규분포 형태로 변환

In [ ]:
df.hist(figsize=(20,15))
plt.show()

In [ ]:
df.sample(10)

In [ ]:
df.describe()

In [ ]:
df.shape

In [ ]:
heat_df = df
heat_df = heat_df.corr().sort_values(by='price', axis=1)

In [ ]:
colormap = plt.cm.plasma_r
plt.figure(figsize = (15, 10))
plt.title('Correlation of Features')
sns.heatmap(heat_df.corr().astype(float), linewidths=0.1, vmax=2.0,
           square=True, cmap=colormap, linecolor='white', annot=True, annot_kws={"size": 12})
plt.show()

- 다중공선성이 존재하는 변수들을 대략적으로 파악

In [ ]:
df.drop('id', axis=1,inplace=True)

In [ ]:
pd.DataFrame(df.groupby('yr_built')['yr_renovated'].value_counts().sort_values())

- yr_built와 yr_renovated를 이용하여 새로운 변수를 추가하기 위해 groupby시도

In [ ]:
df.yr_renovated.value_counts()

In [ ]:
def diff(x):
    if x == 0:
        return 0
    else:
        return 2021-x

df['After_renovated'] = df['yr_renovated'].apply(lambda x: diff(x));df

- 수리된지 얼마의 시간이 지났는지의 변수추가

In [ ]:
plt.figure(figsize = (15, 10))
plt.title('Correlation of Features')
sns.heatmap(df.corr().astype(float), linewidths=0.1, vmax=2.0,
           square=True, cmap=colormap, linecolor='white', annot=True, annot_kws={"size": 12})
plt.show()

- 맨 왼쪽 아래를 보면 새 변수와 Price의 상관관계가 매우 낮은 것을 볼 수 있다.
- 0의 개수가 너무나도 많았기에 좋은 결과를 도출하지 못한 것으로 예상

In [ ]:
df.drop('After_renovated',axis=1, inplace=True)

- 새 변수 삭제

In [ ]:
view_condition = df[['waterfront','view','sqft_above']]
view_condition.hist(figsize=(9,5))

- 각 분포가 왜도를 띄고있고 세 변수 모두 view와 관련된 조건으로 이것들을 이용하여 새로운 변수도출 생각
- 이를 위해 표준화실시
- 음수가 존재하지 않기 때문에 0~1로 변환하는 MinMaxScaler사용

In [ ]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
view_condition_scaled = scaler.fit_transform(view_condition)

view_condition_scaled = pd.DataFrame(data=view_condition_scaled, columns=['waterfront','view','sqft_above'])
view_condition_scaled

In [ ]:
view_condition_scaled.hist()

In [ ]:
df['view_condition'] = view_condition_scaled['waterfront']+ view_condition_scaled['view'] + view_condition_scaled['sqft_above']
df

- 세개의 변수를 합쳐 새 변수 생성

In [ ]:
plt.figure(figsize = (15, 10))
plt.title('Correlation of Features')
sns.heatmap(df.corr().astype(float), linewidths=0.1, vmax=2.0,
           square=True, cmap=colormap, linecolor='white', annot=True, annot_kws={"size": 12})
plt.show()

- 맨 밑쪽을 보면 Price와의 상관관계가 0.58인 것을 알 수 있다.

In [ ]:
plt.figure(figsize=(9,5))
sns.scatterplot(df['view_condition'], df['price'])
sns.regplot(df['view_condition'], df['price'], line_kws={'color':'r'})
plt.title('Correlation : 0.58   Price Vs View_condition')
plt.show()